In [110]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit, when}
import org.apache.spark.sql.types.{IntegerType, DoubleType}

println("all good")

all good


data is in the format:

In [112]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        close: Double,
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt,    s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1),          s(8).split(": ")(1))
}

//val filename = "../Datasets/bitcoin_data_raw.txt"
val filename = "../Datasets/bitcoin_data_max_raw.txt"
val tradingInformation = sc.textFile(filename)
            .flatMap(l => l.split("]}},"))       // Splits into indiviual http calls
            .map(x => x.split(""": \[\{""")(1))  // Splits up between header info and data payload
            .flatMap(x => x.split("},"))         // Now we have row wise data. It is bit messy so need to clean it
            .map(x => x.replace("{", ""))
            .map(x => x.replace("\"", ""))
            .map(x => x.replace("]}}]", ""))     // Last response is special case
            .map(createTradingInfo)
            .cache()

tradingInformation.take(1).foreach(println)
println("All good")

TradingInfo(1595581200,9536.29,9514.4,9514.87,1017.91,9698013.33,9536.26,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_data_max_raw.txt
tradingInformation = MapPartitionsRDD[600] at map at <console>:48


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[600] at map at <console>:48

In [113]:
// Data Action

val df = spark.createDataFrame(tradingInformation).sort("time")
val df1 = df.withColumn("midPrice",($"high" + $"low")/2)

// df1.show(5)

val ratio: Double = 0.9
val training_size = df1.count*ratio
val testing_size = df1.count - 10

val training_position = df1.take(training_size.toInt).last.getInt(0)
val testing_position = df1.take(testing_size.toInt).last.getInt(0)

// val df2 = df1.withColumn("dataset", when(col("time") <= training_position,"training")
//       .when(col("time") > training_position and col("time") <= testing_position,"validation")
//       .otherwise("testing"))
// df2.show(5)


val training_set = df1.filter($"time" <= training_position).cache()
val validation_set = df1.filter($"time" > training_position).cache()//.filter($"time" <= testing_position).cache()
// val testing_set = df1.filter($"time" > testing_position).cache()


// training_set.show(5)
// validation_set.show(5)
// testing_set.show(5)


// println(testing_set.count)

println("All good")

All good


df = [time: int, high: double ... 7 more fields]
df1 = [time: int, high: double ... 8 more fields]
ratio = 0.9
training_size = 12602.7
testing_size = 13993
training_position = 1597734000
testing_position = 1602741600
training_set = [time: int, high: double ... 8 more fields]
validation_set = [time: int, high: double ... 8 more fields]


[time: int, high: double ... 8 more fields]

In [114]:
// training_set
// validation_set
val data_type = "cleaned_data_socialMedia/validation_set"
val dataset = validation_set

val statistics = dataset.agg(mean("midPrice"), stddev("midPrice"), min("midPrice"), max("midPrice"),
                                  mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
                                  mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")).head()

val midPrice_mean = statistics.getDouble(0)
val midPrice_std = statistics.getDouble(1)
val midPrice_min = statistics.getDouble(2)
val midPrice_max = statistics.getDouble(3)
val midPrice_range = midPrice_max - midPrice_min

// // VolumeFrom
val volumefrom_mean = statistics.getDouble(4)
val volumefrom_std = statistics.getDouble(5)
val volumefrom_min = statistics.getDouble(6)
val volumefrom_max = statistics.getDouble(7)
val volumefrom_range = volumefrom_max - volumefrom_min

// // VolumeTo
val volumeto_mean = statistics.getDouble(8)
val volumeto_std = statistics.getDouble(9)
val volumeto_min = statistics.getDouble(10)
val volumeto_max = statistics.getDouble(11)
val volumeto_range = volumeto_max - volumeto_min

// // // from -1 to 1 
// // //val new_dataset = dataset.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// // // from 0 to 1 
val new_dataset = dataset.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
               .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
               .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
               .cache()
               .as("df_2")

new_dataset.show(5)
new_dataset.printSchema()

println("All good")

+----------+--------+--------+--------+----------+-------------+--------+--------------+----------------+------------------+------------------+--------------------+--------------------+
|      time|    high|     low|    open|volumefrom|     volumeto|   close|conversionType|conversionSymbol|          midPrice|      midPriceNorm|      volumefromNorm|        volumetoNorm|
+----------+--------+--------+--------+----------+-------------+--------+--------------+----------------+------------------+------------------+--------------------+--------------------+
|1597737600|12282.35|12254.94| 12264.5|    835.51|1.024674827E7|12256.16|      \direct\|              \\|         12268.645|               1.0|0.010208490625367602|0.011249112124149677|
|1597741200|12262.03|12223.28|12256.16|    951.63|1.164921859E7|12226.21|      \direct\|              \\|         12242.655|0.9886320145041324|0.012100394054222618|0.013213542100509132|
|1597744800| 12293.7|12192.51|12226.21|   1767.11|2.163538782E7|12283.

data_type = cleaned_data_socialMedia/validation_set
dataset = [time: int, high: double ... 8 more fields]
statistics = [10961.180981441823,526.9999478409599,9982.400000000001,12268.645,1536.6393004996435,2851.725433201415,208.94,61586.28,1.688991155802997E7,3.1479601433919687E7,2215642.01,7.161480992E8]
midPrice_mean = 10961.180981441823
midPrice_std = 526.9999478409599
midPrice_min = 9982.400000000001
midPrice_max = 12268.645
midPrice_range = 2286.244999999999
volumefrom_mean = 1536.6393004996435
volumefrom_std = 2851.725433201415
volumefrom_min = 208.94
volumefrom_max = 61586.28
volumefrom_range = 61377.34


volumeto_me...


61377.34

In [ ]:
// Saving data as Text for complete bitcoin timeframe without socialData
// save it 3 times, one for each data set (training / validation / testing )
// don't forget to delete the previous folder 
val pre_processed_dataset = new_dataset.withColumn("empty", typedLit(Seq(1)))
pre_processed_dataset.rdd.repartition(1).saveAsTextFile(data_type)

println("All good")

In [115]:
// Social data is a colelction of various values from different social platforms 
// Explanation of the values can be found at: https://min-api.cryptocompare.com/documentation?key=Social&cat=historicalHourSocialStats
case class SocialData (time: Int,
                        comments: Int,
                        posts: Int,
                        followers: Int,
                        points: Int,
                        overview_page_views: Int,
                        analysis_page_views: Int,
                        markets_page_views: Int,
                        charts_page_views: Int,
                        trades_page_views: Int, 
                        forum_page_views: Int, 
                        influence_page_views: Int, 
                        total_page_views: Int, //
                        fb_likes: Int,
                        fb_talking_about: Int, //
                        twitter_followers: Int,
                        twitter_following: Int,
                        twitter_lists: Int,
                        twitter_favourites: Int,
                        twitter_statuses: Int,
                        reddit_subscribers: Int,
                        reddit_active_users: Int,
                        reddit_posts_per_hour: Double, //
                        reddit_posts_per_day: Double,
                        reddit_comments_per_hour: Double, //
                        reddit_comments_per_day: Double,
                        code_repo_stars: Int,
                        code_repo_forks: Int,
                        code_repo_subscribers: Int,
                        code_repo_open_pull_issues: Int,
                        code_repo_closed_pull_issues: Int,
                        code_repo_open_issues: Int,
                        code_repo_closed_issues: Int,
                        code_repo_contributors: Int                     
                       )

def createSocialData(data: String): SocialData = {
    val s = data.split(", ")
    return SocialData(extractAndConvertToInt(s(0)),    extractAndConvertToInt(s(1)),     extractAndConvertToInt(s(2)),    extractAndConvertToInt(s(3)),
                      extractAndConvertToInt(s(4)),    extractAndConvertToInt(s(5)),     extractAndConvertToInt(s(6)),    extractAndConvertToInt(s(7)),
                      extractAndConvertToInt(s(8)),    extractAndConvertToInt(s(9)),     extractAndConvertToInt(s(10)),   extractAndConvertToInt(s(11)),
                      extractAndConvertToInt(s(12)),   extractAndConvertToInt(s(13)),    extractAndConvertToInt(s(14)),   extractAndConvertToInt(s(15)),
                      extractAndConvertToInt(s(16)),   extractAndConvertToInt(s(17)),    extractAndConvertToInt(s(18)),   extractAndConvertToInt(s(19)),
                      extractAndConvertToInt(s(20)),   extractAndConvertToInt(s(21)),    extractAndConvertToDouble(s(22)),extractAndConvertToDouble(s(23)),
                      extractAndConvertToDouble(s(24)),extractAndConvertToDouble(s(25)), extractAndConvertToInt(s(26)),   extractAndConvertToInt(s(27)),
                      extractAndConvertToInt(s(28)),   extractAndConvertToInt(s(29)),    extractAndConvertToInt(s(30)),   extractAndConvertToInt(s(31)),
                      extractAndConvertToInt(s(32)),   extractAndConvertToInt(s(33)))
}

def extractAndConvertToInt(attribute: String): Int = {
    return attribute.split(": ")(1).toInt
}

def extractAndConvertToDouble(attribute: String): Double = {
    return attribute.split(": ")(1).toDouble
}


val filename = "../Datasets/bitcoin_social_max_raw.txt"
val socialDataInformation = sc.textFile(filename)
            .flatMap(l => l.split("]},"))         // Splits into indiviual http calls
            .map(x => x.split(""": \[\{""")(1))   // Splits up between header info and data payload
            .flatMap(x => x.split("},"))          // Now we have row wise data. It is bit messy so need to clean it
            .map(x => x.replace("{", ""))
            .map(x => x.replace("\"", ""))
            .map(x => x.replace("}", ""))
            .map(x => x.replace("]]", ""))     // Last response is special case
            .map(createSocialData)
            .cache()
    

socialDataInformation.take(1).foreach(println)

println("All good")

SocialData(1595581200,356096,120976,78758,8554310,30013052,1094243,1600267,9444499,800622,9097720,62746,52113149,39654,26,844049,165,6631,1000,20316,1524079,3604,4.4,105.66,51.51,1236.23,44259,26104,3500,386,13298,543,5043,891)
All good


defined class SocialData
filename = ../Datasets/bitcoin_social_max_raw.txt
socialDataInformation = MapPartitionsRDD[671] at map at <console>:93


createSocialData: (data: String)SocialData
extractAndConvertToInt: (attribute: String)Int
extractAndConvertToDouble: (attribute: String)Double


MapPartitionsRDD[671] at map at <console>:93

In [116]:
// Merge TradingInfo with SocialData

val socialDataDf = spark.createDataFrame(socialDataInformation).sort("time")
val socialDatadf_shorten = socialDataDf.select("time", "total_page_views", "fb_talking_about", "reddit_posts_per_hour", "reddit_comments_per_hour")


val statistics = socialDatadf_shorten.agg(mean("total_page_views"), stddev("total_page_views"), min("total_page_views"), max("total_page_views"),
                             mean("fb_talking_about"), stddev("fb_talking_about"), min("fb_talking_about"), max("fb_talking_about"),
                             mean("reddit_posts_per_hour"), stddev("reddit_posts_per_hour"), min("reddit_posts_per_hour"), max("reddit_posts_per_hour"), 
                             mean("reddit_comments_per_hour"), stddev("reddit_comments_per_hour"), min("reddit_comments_per_hour"), max("reddit_comments_per_hour")).head()

val pageViews_mean = statistics.getDouble(0)
val pageViews_std = statistics.getDouble(1)
val pageViews_min = statistics.getInt(2)
val pageViews_max = statistics.getInt(3)
val pageViews_range = pageViews_max - pageViews_min

// // fbTalking
val fbTalking_mean = statistics.getDouble(4)
val fbTalking_std = statistics.getDouble(5)
val fbTalking_min = statistics.getInt(6)
val fbTalking_max = statistics.getInt(7)
val fbTalking_range = fbTalking_max - fbTalking_min

// // redditPosts
val redditPosts_mean = statistics.getDouble(8)
val redditPosts_std = statistics.getDouble(9)
val redditPosts_min = statistics.getDouble(10)
val redditPosts_max = statistics.getDouble(11)
val redditPosts_range = redditPosts_max - redditPosts_min

val redditComments_mean = statistics.getDouble(12)
val redditComments_std = statistics.getDouble(13)
val redditComments_min = statistics.getDouble(14)
val redditComments_max = statistics.getDouble(15)
val redditComments_range = redditComments_max - redditComments_min


val socialDatadf_shorten_norm = socialDatadf_shorten.withColumn("pageViewsNorm",(($"total_page_views" - pageViews_min)/pageViews_range).cast(DoubleType))
               .withColumn("fbTalkingNorm",(($"fb_talking_about" - fbTalking_min)/fbTalking_range).cast(DoubleType))
               .withColumn("redditPostsNorm",(($"reddit_posts_per_hour" - redditPosts_min)/redditPosts_range).cast(DoubleType))
               .withColumn("redditCommentsNorm",(($"reddit_comments_per_hour" - redditComments_min)/redditComments_range).cast(DoubleType))
               .cache()

val mergedDataDf = new_dataset.join(socialDatadf_shorten_norm, Seq("time")).cache()
mergedDataDf.printSchema()

root
 |-- time: integer (nullable = false)
 |-- high: double (nullable = false)
 |-- low: double (nullable = false)
 |-- open: double (nullable = false)
 |-- volumefrom: double (nullable = false)
 |-- volumeto: double (nullable = false)
 |-- close: double (nullable = false)
 |-- conversionType: string (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- midPrice: double (nullable = true)
 |-- midPriceNorm: double (nullable = true)
 |-- volumefromNorm: double (nullable = true)
 |-- volumetoNorm: double (nullable = true)
 |-- total_page_views: integer (nullable = false)
 |-- fb_talking_about: integer (nullable = false)
 |-- reddit_posts_per_hour: double (nullable = false)
 |-- reddit_comments_per_hour: double (nullable = false)
 |-- pageViewsNorm: double (nullable = true)
 |-- fbTalkingNorm: double (nullable = true)
 |-- redditPostsNorm: double (nullable = true)
 |-- redditCommentsNorm: double (nullable = true)



socialDataDf = [time: int, comments: int ... 32 more fields]
socialDatadf_shorten = [time: int, total_page_views: int ... 3 more fields]
statistics = [4.777035968592445E7,3771379.0518834656,40416602,53304506,38.02677997571949,15.012885379980952,0,78,5.204202670856242,2.686563409250299,0.3,96.8,98.60628651003351,56.19267435160475,23.5,1457.49]
pageViews_mean = 4.777035968592445E7
pageViews_std = 3771379.0518834656
pageViews_min = 40416602
pageViews_max = 53304506
pageViews_range = 12887904
fbTalking_mean = 38.02677997571949
fbTalking_std = 15.012885379980952
fbTalking_min = 0
fbTalking_max = 78
fbTalking_range = 78...


78...

In [20]:
// Saving data as Text

// don't forget to delete the previous folder 
val finalDf = mergedDataDf.withColumn("empty", typedLit(Seq(1)))
finalDf.rdd.repartition(1).saveAsTextFile(data_type)

println("All good")


All good


finalDf = [time: int, high: double ... 20 more fields]


[time: int, high: double ... 20 more fields]